# View of SQL

In [84]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

In [85]:
import pandas as pd
import sqlalchemy as sql
import pyodbc

server = 'sql-cl-dw-pro\datawarehouse'
driver = 'SQL+Server'
db = 'ODS_CPS'
myQuery = '''
with cte as
(
select studentID,SchoolYear, test,date,
CASE 
     WHEN level = 'Level 1: Unsatisfactory' or level = 'Does Not Meet Grade Level' 
               THEN 0
               ELSE 1
       END as ifPass, subject, 
row_number()over(partition by schoolyear, subject, studentID order by date) as rn
from rpt.vStudentAssessment
 where test like'%STAAR%'and level is not null
 and (Subject='Math' or Subject='reading')
)

select distinct c.SchoolYear, vs.studentID, c.ifPass, c.Subject, vs.SchoolID, vs.SchoolName, vs.SchoolNameAbbreviated,c.date
from cte c
inner join RPT.vStudent vs
on c.StudentID=vs.StudentID and c.SchoolYear=vs.SchoolYear
where rn=1
order by vs.studentID
'''
engine = sql.create_engine('mssql+pyodbc://{}/{}?driver={}'.format(server, db, driver))
df = pd.read_sql_query(myQuery, engine)

In [86]:
df.sample(10)

,SchoolYear,studentID,ifPass,Subject,SchoolID,SchoolName,SchoolNameAbbreviated,date
21149,2013-2014,816102509.0,1,Math,220816101,Uplift Summit Prep PS,Summit PS,2014-04-01
63127,2012-2013,843101961.0,0,Reading,57843041,Uplift Hampton Prep MS,Hampton MS,2013-04-01
48814,2017-2018,842101548.0,1,Reading,57803046,Uplift Heights Prep MS,Heights MS,2018-05-01
9949,2015-2016,803104466.0,1,Math,57803041,Uplift North Hills Prep MS,North Hills MS,2016-05-01
64203,2016-2017,843102409.0,1,Reading,57803103,Uplift Gradus Prep PS,Gradus PS,2017-05-01
17177,2017-2018,816101010.0,1,Reading,57803047,Uplift Summit Prep MS,Summit MS,2018-05-01
36309,2013-2014,838102248.0,0,Math,57838001,Uplift Peak Prep PS,Peak PS,2014-04-01
11746,2016-2017,803104987.0,1,Reading,57803043,Uplift Infinity Prep MS,Infinity MS,2017-03-01
28939,2017-2018,816105541.0,1,Reading,57803047,Uplift Summit Prep MS,Summit MS,2018-04-01
30510,2011-2012,838100710.0,1,Reading,57838002,Uplift Peak Prep MS,Peak MS,2012-03-01


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78676 entries, 0 to 78675
Data columns (total 8 columns):
SchoolYear               78676 non-null object
studentID                78676 non-null float64
ifPass                   78676 non-null int64
Subject                  78676 non-null object
SchoolID                 78676 non-null int64
SchoolName               78676 non-null object
SchoolNameAbbreviated    78676 non-null object
date                     78676 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 4.8+ MB


In [102]:

e = df.groupby(["SchoolYear","SchoolName","Subject",'ifPass'])['ifPass'].count().rename("count")

SchoolYear  SchoolName                               Subject  ifPass
2011-2012   Uplift Hampton Prep MS                   Math     0          71
                                                              1         123
                                                     Reading  0          70
                                                              1         215
            Uplift Hampton Prep PS                   Math     0          25
                                                              1          29
                                                     Reading  0          20
                                                              1          34
            Uplift Heights Prep MS                   Math     0          49
                                                              1         170
                                                     Reading  0          54
                                                              1         165
            Uplift 

In [131]:
df2 = df.groupby(["SchoolYear","SchoolName","Subject",'ifPass']).agg({'ifPass': 'count'})


In [132]:
df2

ifPass
SchoolYear SchoolName                              Subject ifPass        
2011-2012  Uplift Hampton Prep MS                  Math    0           71
                                                           1          123
                                                   Reading 0           70
                                                           1          215
           Uplift Hampton Prep PS                  Math    0           25
                                                           1           29
                                                   Reading 0           20
                                                           1           34
           Uplift Heights Prep MS                  Math    0           49
                                                           1          170
                                                   Reading 0           54
                                                           1          165
           Uplift Infinity Prep MS                 Math    0           15
                                                           1          108
                                                   Reading 0           22
                                                           1          101
           Uplift Luna Prep MS                     Math    0            8
                                                           1           79
                                                   Reading 0           20
                                                           1           67
           Uplift Luna Prep PS                     Math    0           11
                                                           1           30
                                                   Reading 0            7
                                                           1           34
           Uplift North Hills Prep MS              Math    0            9
                                                           1          440
                                                   Reading 0           14
                                                           1          435
           Uplift North Hills Prep PS              Math    0           19
                                                           1          255
...                                                                   ...
2017-2018  Uplift Pinnacle Secondary Summer School Reading 0           77
                                                           1           56
           Uplift Summit Prep MS                   Math    0           82
                                                           1          234
                                                   Reading 0          115
                                                           1          350
           Uplift Summit Prep PS                   Math    0           58
                                                           1          241
                                                   Reading 0           83
                                                           1          216
           Uplift Summit Summer School             Math    0          151
                                                           1          136
                                                   Reading 0          234
                                                           1          171
           Uplift Triumph Prep PS                  Math    0           62
                                                           1          207
                                                   Reading 0           90
                                                           1          179
           Uplift Williams Prep MS                 Math    0           27
                                                           1          232
                                                   Reading 0           58
                                                           1          3

In [143]:
# Change: groupby state_office and divide by sum
Passingpercentage = df2.groupby(level=[0,1,2]).apply(lambda x:
                                                 100 * x / float(x.sum()))
#float(x.sum()))


In [144]:
Passingpercentage

ifPass
SchoolYear SchoolName                              Subject ifPass           
2011-2012  Uplift Hampton Prep MS                  Math    0       36.597938
                                                           1       63.402062
                                                   Reading 0       24.561404
                                                           1       75.438596
           Uplift Hampton Prep PS                  Math    0       46.296296
                                                           1       53.703704
                                                   Reading 0       37.037037
                                                           1       62.962963
           Uplift Heights Prep MS                  Math    0       22.374429
                                                           1       77.625571
                                                   Reading 0       24.657534
                                                           1       75.342466
           Uplift Infinity Prep MS                 Math    0       12.195122
                                                           1       87.804878
                                                   Reading 0       17.886179
                                                           1       82.113821
           Uplift Luna Prep MS                     Math    0        9.195402
                                                           1       90.804598
                                                   Reading 0       22.988506
                                                           1       77.011494
           Uplift Luna Prep PS                     Math    0       26.829268
                                                           1       73.170732
                                                   Reading 0       17.073171
                                                           1       82.926829
           Uplift North Hills Prep MS              Math    0        2.004454
                                                           1       97.995546
                                                   Reading 0        3.118040
                                                           1       96.881960
           Uplift North Hills Prep PS              Math    0        6.934307
                                                           1       93.065693
...                                                                      ...
2017-2018  Uplift Pinnacle Secondary Summer School Reading 0       57.894737
                                                           1       42.105263
           Uplift Summit Prep MS                   Math    0       25.949367
                                                           1       74.050633
                                                   Reading 0       24.731183
                                                           1       75.268817
           Uplift Summit Prep PS                   Math    0       19.397993
                                                           1       80.602007
                                                   Reading 0       27.759197
                                                           1       72.240803
           Uplift Summit Summer School             Math    0       52.613240
                                                           1       47.386760
                                                   Reading 0       57.777778
                                                           1       42.222222
           Uplift Triumph Prep PS                  Math    0       23.048327
                                                           1       76.951673
                                                   Reading 0       33.457249
                                                           1       66.542751
           Uplift Williams Prep MS                 Math    0       10.424710
                                                           1      

In [145]:
Passingpercentage.to_csv('C:/Users/jli/Documents/Python Scripts/passPercentage', encoding='utf-8', index=False)

In [148]:
from pandas import ExcelWriter
writer = ExcelWriter('C:/Users/jli/Documents/Python Scripts/passPercentage.xlsx')
Passingpercentage.to_excel(writer,'Sheet1')
writer.save()

In [150]:
import os
cwd = os.getcwd()

In [151]:
cwd

'C:\\Users\\jli'